# Instituto Tecnológico de Costa Rica

## Programa de Ciencia de los Datos

### Curso: Estadística para Ciencia de los Datos

Tarea #1

Estudiante:


> Natalia Rojas Canales

Profesor:

> Felipe Meza

Noviembre 2019

# Instrucciones
Conseguir un .csv interesante y hacer un notebook utilizando 5 Insight de los utilizados en clase y 3 nuevos.

### Pre-requisitos

- findspark
- pyspark
- spark

In [1]:
!pip install pyspark


  Found existing installation: py4j 0.10.7
    Uninstalling py4j-0.10.7:
      Successfully uninstalled py4j-0.10.7


In [2]:
!pip install findspark

In [3]:
!pip install spark

Una vez instaladas las librerias necesarias, comenzamos con la importación de librerías y demás parámetros requeridos.

In [4]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('data_processing').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.types import *

In [5]:
import findspark
#findspark.init('/opt/spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType



In [6]:
#seleccione un arhivo de datos de suicidios
df=spark.read.csv("suicidios.csv",header=True, inferSchema=True)

In [7]:
df.printSchema() 
#ver el esquema del archivo de datos

root
 |-- country: string (nullable = true)
 |-- year1: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- suicides_no: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- suicides/100k pop: double (nullable = true)
 |-- Year7: integer (nullable = true)
 |-- HDI for year: double (nullable = true)
 |--  gdp_for_year ($) : string (nullable = true)
 |-- gdp_per_capita ($): integer (nullable = true)
 |-- generation: string (nullable = true)



In [17]:
df.count() #cantidad de datos en el archivo

27820

In [18]:

len(df.columns) #cantidd de columnas en el archivo de datos

12

In [19]:
df.show() #una mirada rápida a los datos, dado que no se mostrarán todos

+-------+-----+------+-----------+-----------+----------+-----------------+-----+------------+------------------+------------------+---------------+
|country|year1|   sex|        age|suicides_no|population|suicides/100k pop|Year7|HDI for year| gdp_for_year ($) |gdp_per_capita ($)|     generation|
+-------+-----+------+-----------+-----------+----------+-----------------+-----+------------+------------------+------------------+---------------+
|Albania| 1987|  male|15-24 years|         21|    312900|             6.71| 1987|        null|     2,156,624,900|               796|   Generation X|
|Albania| 1987|  male|35-54 years|         16|    308000|             5.19| 1987|        null|     2,156,624,900|               796|         Silent|
|Albania| 1987|female|15-24 years|         14|    289700|             4.83| 1987|        null|     2,156,624,900|               796|   Generation X|
|Albania| 1987|  male|  75+ years|          1|     21800|             4.59| 1987|        null|     2,156,6

In [22]:
 #estos datos son de suicidios  y están registrados por país, así que me parece interesante ver los datos agrupados por esa característica
df.groupBy('country').count().show()

+--------------------+-----+
|             country|count|
+--------------------+-----+
|             Albania|  264|
| Antigua and Barbuda|  324|
|           Argentina|  372|
|             Armenia|  298|
|               Aruba|  168|
|           Australia|  360|
|             Austria|  382|
|          Azerbaijan|  192|
|             Bahamas|  276|
|             Bahrain|  252|
|            Barbados|  300|
|             Belarus|  252|
|             Belgium|  372|
|              Belize|  336|
|Bosnia and Herzeg...|   24|
|              Brazil|  372|
|            Bulgaria|  360|
|          Cabo Verde|   12|
|              Canada|  348|
|               Chile|  372|
+--------------------+-----+
only showing top 20 rows



In [27]:
#pero antes solo vi la cantidad de registros, realmente quiero ver la cantidad de sucicidios, pero agrupados por sexo
df.groupBy('sex').sum('suicides_no').show()

+------+----------------+
|   sex|sum(suicides_no)|
+------+----------------+
|  male|         5188910|
|female|         1559510|
+------+----------------+



##### Que  MONTON de hombres!!! 

In [28]:
#pero ahora vamos a agrupar por país, sexo y rando de edad
df.groupBy('country', 'sex', 'age').sum('suicides_no').show()

+-------------------+------+-----------+----------------+
|            country|   sex|        age|sum(suicides_no)|
+-------------------+------+-----------+----------------+
|            Albania|  male|15-24 years|             293|
|            Albania|  male|35-54 years|             418|
|            Albania|female|15-24 years|             253|
|            Albania|  male|  75+ years|              42|
|            Albania|  male|25-34 years|             288|
|            Albania|female|  75+ years|              39|
|            Albania|female|35-54 years|             173|
|            Albania|female|25-34 years|             139|
|            Albania|  male|55-74 years|             196|
|            Albania|female| 5-14 years|              21|
|            Albania|female|55-74 years|              68|
|            Albania|  male| 5-14 years|              40|
|Antigua and Barbuda|female|15-24 years|               0|
|Antigua and Barbuda|female|25-34 years|               0|
|Antigua and B

In [25]:
#pero ahora vamos a agrupar por país, sexo y rando 
df.groupBy('country', 'sex', 'age').sum('suicides_no').show()

+-------------------+------+-----------+----------------+
|            country|   sex|        age|sum(suicides_no)|
+-------------------+------+-----------+----------------+
|            Albania|  male|15-24 years|             293|
|            Albania|  male|35-54 years|             418|
|            Albania|female|15-24 years|             253|
|            Albania|  male|  75+ years|              42|
|            Albania|  male|25-34 years|             288|
|            Albania|female|  75+ years|              39|
|            Albania|female|35-54 years|             173|
|            Albania|female|25-34 years|             139|
|            Albania|  male|55-74 years|             196|
|            Albania|female| 5-14 years|              21|
|            Albania|female|55-74 years|              68|
|            Albania|  male| 5-14 years|              40|
|Antigua and Barbuda|female|15-24 years|               0|
|Antigua and Barbuda|female|25-34 years|               0|
|Antigua and B

In [36]:
# analizar valores asociados a la columna coutry en forma de estadistica de media 
df.groupBy('country').agg(F.mean('suicides_no')).show() 

+--------------------+--------------------+
|             country|    avg(suicides_no)|
+--------------------+--------------------+
|             Albania|   7.462121212121212|
| Antigua and Barbuda|0.033950617283950615|
|           Argentina|   221.0188172043011|
|             Armenia|    6.39261744966443|
|               Aruba|  0.6011904761904762|
|           Australia|  194.75277777777777|
|             Austria|  131.08115183246073|
|          Azerbaijan|               8.625|
|             Bahamas| 0.33695652173913043|
|             Bahrain|  1.8373015873015872|
|            Barbados|                0.59|
|             Belarus|  237.66666666666666|
|             Belgium|  168.71236559139786|
|              Belize|  1.0357142857142858|
|Bosnia and Herzeg...|               13.25|
|              Brazil|   609.1747311827957|
|            Bulgaria|  101.07777777777778|
|          Cabo Verde|                 3.5|
|              Canada|   309.0833333333333|
|               Chile|  109.9327